In [70]:
import numpy as np
import pandas as pd
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Import Data

In [22]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

X_train = train.drop('SalePrice', axis = 1)
X_test = test
y_train = train['SalePrice']

## Combine train and test data

In [23]:
# 合并一下train和test，两个一起处理，之后再分开
X = X_train.append(X_test, ignore_index = True)
# Drop id
X.drop('Id', axis = 1, inplace = True)

print(X_train.shape, X_test.shape, X.shape)
X.head()

(1460, 80) (1459, 80) (2919, 79)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [24]:
# 0 - 1459 是train
# 1460 开始是test

## Numerical columns and Categorical columns

In [25]:
# Numerical Columns
num_col = X.columns[(X.dtypes != 'object').values.tolist()].tolist()
cat_col = X.columns[(X.dtypes == 'object').values.tolist()].tolist()

In [26]:
# Check if we catch every column
print('num_col:', len(num_col))
print('cat_col:', len(cat_col))
print('X:', len(X.columns))

num_col: 36
cat_col: 43
X: 79


# Data preprocessing

## Missingness

In [27]:
missing = X.isnull().sum(axis = 0)
missing = (missing[missing != 0])/X.shape[0]
pd.DataFrame(missing, columns = ['Missing Percent']).sort_values('Missing Percent', ascending = False)

,Missing Percent
PoolQC,0.996574
MiscFeature,0.964029
Alley,0.932169
Fence,0.804385
FireplaceQu,0.486468
LotFrontage,0.166495
GarageFinish,0.054471
GarageQual,0.054471
GarageCond,0.054471
GarageYrBlt,0.054471


### Impute missingness Type 1 (miss是因为没有）

In [28]:
# Alley
X['Alley'].fillna('No_Alley', inplace = True)

# Bsmt相关
Bsmt_col = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
for i in Bsmt_col:
    X[i].fillna('No_Bsmt', inplace = True)

# Garage相关
Garage_col = ['GarageType','GarageFinish', 'GarageQual', 'GarageCond']
for k in Garage_col:
    X[k].fillna('No_Garage', inplace = True)
    
Garage_col2 = ['GarageYrBlt','GarageCars', 'GarageArea']
for a in Garage_col2:
    X[a].fillna(0, inplace = True)

# FireplaceQu
X['FireplaceQu'].fillna('No_Fireplace', inplace = True)

# PoolQC
X['PoolQC'].fillna('No_Pool', inplace = True)

# Fence
X['Fence'].fillna('No_Fence', inplace = True)

# MiscFeature
X['MiscFeature'].fillna('No_MiscFeature', inplace = True)

In [29]:
# Check missingness again
missing = X.isnull().sum(axis = 0)
missing = (missing[missing != 0])/X.shape[0]
pd.DataFrame(missing, columns = ['Missing Percent']).sort_values('Missing Percent', ascending = False)

,Missing Percent
LotFrontage,0.166495
MasVnrType,0.008222
MasVnrArea,0.007879
MSZoning,0.001370
Utilities,0.000685
Functional,0.000685
BsmtHalfBath,0.000685
BsmtFullBath,0.000685
Electrical,0.000343
KitchenQual,0.000343


### Impute missingness Type 2 (median)

In [30]:
missing_col = missing.index.tolist()
num_missing_col = []
for i in missing_col:
    if i in num_col:
        num_missing_col.append(i)
num_missing_col

['LotFrontage',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'BsmtFullBath',
 'BsmtHalfBath']

In [31]:
for i in num_missing_col:
    X[i] = X.groupby(['Neighborhood'])[i].apply(lambda x: x.fillna(x.median()))

### Impute missingness Type 3 (mode)

In [32]:
missing_col = missing.index.tolist()
cat_missing_col = []
for i in missing_col:
    if i in cat_col:
        cat_missing_col.append(i)
cat_missing_col

['MSZoning',
 'Utilities',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'Electrical',
 'KitchenQual',
 'Functional',
 'SaleType']

In [33]:
for i in cat_missing_col:
    X[i] = X.groupby(['Neighborhood'])[i].apply(lambda x:x.fillna(x.value_counts().index[0]))

### Check missingness again

In [34]:
X.isnull().sum().sum()

0

In [35]:
# OK everything clear!

## Nominal / Ordinal

### Onehot

In [36]:
onehot_col = ['MSSubClass','MSZoning','Street','Alley','LotShape','LandContour','Utilities','LotConfig','Neighborhood',
              'Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st',
              'Exterior2nd','MasVnrType','Foundation','Heating','CentralAir','Electrical','GarageType',
              'MiscFeature','SaleType','SaleCondition']

In [37]:
X = pd.get_dummies(X, columns = onehot_col, drop_first=True)

In [38]:
X.head()

,LotFrontage,LotArea,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,65.0,8450,Gtl,7,5,2003,2003,196.0,Gd,TA,...,0,0,0,0,1,0,0,0,1,0
1,80.0,9600,Gtl,6,8,1976,1976,0.0,TA,TA,...,0,0,0,0,1,0,0,0,1,0
2,68.0,11250,Gtl,7,5,2001,2002,162.0,Gd,TA,...,0,0,0,0,1,0,0,0,1,0
3,60.0,9550,Gtl,7,5,1915,1970,0.0,TA,TA,...,0,0,0,0,1,0,0,0,0,0
4,84.0,14260,Gtl,8,5,2000,2000,350.0,Gd,TA,...,0,0,0,0,1,0,0,0,1,0


### Order

In [39]:
X['LandSlope'].replace({'Sev':2, 'Mod':1, 'Gtl':0}, inplace = True)

X['BsmtExposure'].replace({'Gd':4, 'Av':3, 'Mn':2, 'No':1, 'No_Bsmt':0}, inplace = True)

X['Functional'].replace({'Typ':7, 'Min1':6, 'Min2':5, 'Mod':4, 'Maj1':3, 
                        'Maj2':2, 'Sev':1, 'Sal':0}, inplace = True)

X['FireplaceQu'].replace({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'No_Fireplace':0}, inplace = True)

X['GarageFinish'].replace({'Fin':3, 'RFn':2, 'Unf':1, 'No_Garage':0}, inplace = True)

X['PavedDrive'].replace({'Y':2, 'P':1, 'N':0}, inplace = True)

X['PoolQC'].replace({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'No_Pool':0}, inplace = True)

X['Fence'].replace({'GdPrv':4, 'MnPrv':3, 'GdWo':2, 'MnWw':1, 'No_Fence':0}, inplace = True)

replace_dict1 = {'Ex':4, 'Gd':3, 'TA':2, 'Fa':1, 'Po':0}
type1_col = ['ExterQual', 'ExterCond', 'HeatingQC','KitchenQual']
for i in type1_col:
    X[i].replace(replace_dict1, inplace = True)

replace_dict2 = {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'No_Bsmt':0}
type2_col = ['BsmtQual', 'BsmtCond']
for i in type2_col:
    X[i].replace(replace_dict2, inplace = True)

replace_dict3 = {'GLQ':6, 'ALQ':5, 'BLQ':4, 'Rec':3, 'LwQ':2, 'Unf':1, 'No_Bsmt':0}
type3_col = ['BsmtFinType1','BsmtFinType2']
for i in type3_col:
    X[i].replace(replace_dict3, inplace = True)

replace_dict4 = {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'No_Garage':0}
type4_col = ['GarageQual','GarageCond']
for i in type4_col:
    X[i].replace(replace_dict4, inplace = True)

In [40]:
# Check if we convert everything to numerical values
(X.dtypes).loc[(X.dtypes == object).tolist()]


Series([], dtype: object)

In [41]:
# Everything clear!

# Feature Engineering

In [44]:
print(X.shape)
X.head()

(2919, 218)


,LotFrontage,LotArea,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,65.0,8450,0,7,5,2003,2003,196.0,3,2,...,0,0,0,0,1,0,0,0,1,0
1,80.0,9600,0,6,8,1976,1976,0.0,2,2,...,0,0,0,0,1,0,0,0,1,0
2,68.0,11250,0,7,5,2001,2002,162.0,3,2,...,0,0,0,0,1,0,0,0,1,0
3,60.0,9550,0,7,5,1915,1970,0.0,2,2,...,0,0,0,0,1,0,0,0,0,0
4,84.0,14260,0,8,5,2000,2000,350.0,3,2,...,0,0,0,0,1,0,0,0,1,0


### Generate new features

In [47]:
################## 所有关于时间的 ##################
X['Sell_from_Build'] = X['YrSold'] - X['YearBuilt']
X['Sell_from_Remod'] = X['YrSold'] - X['YearRemodAdd']
X['Sell_from_Garage'] = X['YrSold'] - X['GarageYrBlt']
X['Remod_from_Build'] = X['YearRemodAdd'] - X['YearBuilt']
X['Garage_from_Build'] = X['GarageYrBlt'] - X['YearBuilt']
# 注意那个GarageYrBlt没有Garage的就给我弄成0了！！

# Avoid negative number for ages
for i in ['Sell_from_Build','Sell_from_Remod','Sell_from_Garage','Remod_from_Build','Garage_from_Build']:
    X[i] = X[i].map(lambda x: 0 if x < 0 else x)

# Update num_col
num_col += ['Sell_from_Build','Sell_from_Remod','Sell_from_Garage','Remod_from_Build','Garage_from_Build']

In [50]:
################## 所有关于Bath的 ##################
X['BsmtBath'] = X['BsmtFullBath'] + 0.5 * X['BsmtHalfBath']
X['Bath'] = X['FullBath'] + 0.5 * X['HalfBath']    

In [51]:
################## 所有Cond+Qual ##################
X['Overall'] = X['OverallCond'] + X['OverallQual']
X['Exter'] = X['ExterCond'] + X['ExterQual']
X['Bsmt'] = X['BsmtCond'] + X['BsmtQual']
X['Garage'] = X['GarageQual'] + X['GarageCond']

In [52]:
print(X.shape)
X.head()

(2919, 229)


,LotFrontage,LotArea,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,...,Sell_from_Remod,Sell_from_Garage,Remod_from_Build,Garage_from_Build,BsmtBath,Bath,Overall,Exter,Bsmt,Garage
0,65.0,8450,0,7,5,2003,2003,196.0,3,2,...,5,5.0,0,0.0,1.0,2.5,12,5,7,6
1,80.0,9600,0,6,8,1976,1976,0.0,2,2,...,31,31.0,0,0.0,0.5,2.0,14,4,7,6
2,68.0,11250,0,7,5,2001,2002,162.0,3,2,...,6,7.0,1,0.0,1.0,2.5,12,5,7,6
3,60.0,9550,0,7,5,1915,1970,0.0,2,2,...,36,8.0,55,83.0,1.0,1.0,12,4,7,6
4,84.0,14260,0,8,5,2000,2000,350.0,3,2,...,8,8.0,0,0.0,1.0,2.5,13,5,7,6


### Skewness 

In [59]:
num_col.remove('MSSubClass')

In [63]:
col_name = []
col_skew = []
for i in num_col:
    col_name.append(i)
    col_skew.append(X[i].skew())
skew_df = pd.DataFrame({'Column':col_name, 'Skewness':col_skew}).sort_values('Skewness', ascending = False)
skew_df.head()

,Column,Skewness
32,MiscVal,21.958480
31,PoolArea,16.907017
1,LotArea,12.829025
13,LowQualFinSF,12.094977
29,3SsnPorch,11.381914


In [66]:
skewed_features = skew_df[abs(skew_df['Skewness']) >= 0.5]['Column'].tolist()


In [68]:
X[skewed_features] = np.log1p(X[skewed_features])

### StandardSclaer

In [71]:
from sklearn import preprocessing
sd = preprocessing.StandardScaler()

In [72]:
X[num_col] = sd.fit_transform(X[num_col])

In [75]:
print(X.shape)
X.head()

(2919, 229)


,LotFrontage,LotArea,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,...,Sell_from_Remod,Sell_from_Garage,Remod_from_Build,Garage_from_Build,BsmtBath,Bath,Overall,Exter,Bsmt,Garage
0,-0.052108,-0.103719,0,0.646183,-0.435439,1.040634,0.896833,1.217601,3,2,...,-0.887142,-0.804089,-0.738006,-0.413670,1.0,2.5,12,5,7,6
1,0.570217,0.146544,0,-0.063185,1.906370,0.161526,-0.395604,-0.799566,2,2,...,0.357658,0.211138,-0.738006,-0.413670,0.5,2.0,14,4,7,6
2,0.082971,0.457629,0,0.646183,-0.435439,0.975922,0.848965,1.145266,3,2,...,-0.839265,-0.629617,-0.310935,-0.413670,1.0,2.5,12,5,7,6
3,-0.291506,0.136301,0,0.646183,-0.435439,-1.869638,-0.682812,-0.799566,2,2,...,0.597043,-0.558184,1.742149,3.353037,1.0,1.0,12,4,7,6
4,0.716693,0.922662,0,1.355551,-0.435439,0.943542,0.753229,1.438126,3,2,...,-0.743511,-0.558184,-0.738006,-0.413670,1.0,2.5,13,5,7,6


### Delete correlated columns

In [107]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

corr_features = correlation(X, 0.8)
print('correlated features: ', len(set(corr_features)) )

correlated features:  35


In [109]:
X.drop(labels = corr_features, axis = 1, inplace = True)

## Back to Train and Test

In [110]:
X_train = X[:1460]
X_test = X[1460:]
y_train = np.log(y_train)

In [111]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(2919, 194)
(1460, 194)
(1459, 194)


In [112]:
t = ensemble.RandomForestRegressor()
t.fit(X_train, y_train)

RandomForestRegressor()

In [121]:
sum(t.feature_importances_[:35])

0.9100733578012467

# Feature Selection

In [125]:
from sklearn import ensemble
from sklearn import feature_selection
from mlxtend.feature_selection import SequentialFeatureSelector as SFS


sfs = SFS(
    ensemble.RandomForestRegressor(n_jobs = -1),
    k_features = 16,
    forward = True,
    floating = False,
    scoring = 'r2',
    verbose = 2,
    cv = 5,
    n_jobs = -1
)
sfs = sfs.fit(X_train.to_numpy(), y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   32.3s
[Parallel(n_jobs=-1)]: Done 194 out of 194 | elapsed:   41.5s finished

[2020-08-31 21:16:19] Features: 1/16 -- score: 0.664508439052406[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   34.8s
[Parallel(n_jobs=-1)]: Done 193 out of 193 | elapsed:   44.7s finished

[2020-08-31 21:17:04] Features: 2/16 -- score: 0.710760350169798[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   35.8s
[Parallel(n_jobs=-1)]: Done 192 out of 192 | elapsed:   45.7s finished

[2020-08-31 21:17:50] Features: 3/16 -- score: 0.7399043

In [128]:
# Get the selected feature
selected_feat = X_train.columns[list(sfs.k_feature_idx_)].tolist()
selected_feat

['LotArea',
 'LandSlope',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'BsmtExposure',
 'BsmtFinType1',
 '1stFlrSF',
 'GrLivArea',
 'Fireplaces',
 'GarageFinish',
 'GarageCars',
 'MSZoning_RL',
 'Heating_OthW',
 'CentralAir_Y',
 'Overall']

# Train Model

In [138]:
y_train = np.log(train['SalePrice'])

In [139]:
X_train_selected = X_train[selected_feat]
X_test_selected = X_test[selected_feat]

In [140]:
rf = ensemble.RandomForestRegressor()
rf.fit(X_train_selected, y_train)

RandomForestRegressor()

In [148]:
prediction = np.exp(rf.predict(X_test_selected))

In [149]:
result = pd.DataFrame({'Id': test['Id'], 'SalePrice': prediction})

In [151]:
# result.to_csv('result.csv',index = False)